# Определение похожих документов

#### Цели работы

* Освоить интерфейс TfidfVectorizer.
* Закрепить навыки токенизации текста.
* Закрепить навыки удаления стоп-слов и лемматизации текста.
* Научиться векторизовать текст.

### Что нужно сделать
В файле `vacansies.csv` — сотни разных вакансий, некоторые из них — похожи. Найдите вакансию с наибольшим числом похожих вакансий. Считайте, что вакансии похожи, если косинусное расстояние между векторами, которые представлют их тексты, меньше `0.5`.


###Установка библиотек, подключение диска

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

### Импорт библиотек

In [2]:
import pymorphy2
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

### Загрузка датасета

In [3]:
data = pd.read_csv(f'data/vacancies.csv')
data.head()

,text
0,Старший Java-разработчик в Музыку🎧\n\nВас ждет...
1,Python-разработчик в Яндекс.Лавку🍔\n\nЯндекс.Л...
2,Фронтенд-разработчик в Вертикали🏠\n\nВертикали...
3,iOS-разработчик в Вертикали (Буткемп)🍏\n\nВерт...
4,Старший разработчик в группу разработки бессер...


### Создание функции препроцессора для использования в векторизации

In [4]:
def preprocess(text):
    lemm_words = []
    t = RegexpTokenizer('\w+')
    m = pymorphy2.MorphAnalyzer()
    s = stopwords.words('russian')
    for word in t.tokenize(text):
        word = word.lower()
        if word not in s:
            lemm_words.append(m.parse(word)[0].normal_form)
    return ' '.join(lemm_words)

### Векторизация и расчет расстояний ###

In [5]:
vectorizer = TfidfVectorizer(preprocessor=preprocess, norm=None)
tfidf_matrix = vectorizer.fit_transform(data['text'])

In [6]:
result = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
dist = cosine_distances(result)
vac = pd.DataFrame(dist)
vac

,0,1,2,3,4,5,6,7,8,9,...,615,616,617,618,619,620,621,622,623,624
0,0.000000,0.867277,0.892791,0.898116,0.890252,0.924523,0.863116,0.004905,0.937734,0.744903,...,0.969723,0.962532,0.948985,0.937083,0.958178,0.852990,0.901102,0.950424,9.406659e-01,9.406659e-01
1,0.867277,0.000000,0.850028,0.883378,0.932432,0.910260,0.872761,0.866623,0.851844,0.941676,...,0.942066,0.930739,0.945752,0.966840,0.940882,0.868696,0.930068,0.851514,8.730193e-01,8.730193e-01
2,0.892791,0.850028,0.000000,0.705472,0.956243,0.895629,0.910963,0.910157,0.919442,0.954458,...,0.782189,0.694877,0.892991,0.798895,0.957063,0.945965,0.944117,0.950632,9.160423e-01,9.160423e-01
3,0.898116,0.883378,0.705472,0.000000,0.953029,0.922851,0.918541,0.897613,0.956812,0.953168,...,0.784778,0.886813,0.906069,0.974369,0.950082,0.937717,0.952347,0.966035,9.722551e-01,9.722551e-01
4,0.890252,0.932432,0.956243,0.953029,0.000000,0.911678,0.921108,0.889711,0.939244,0.929367,...,0.976503,0.975582,0.965005,0.937016,0.966852,0.897681,0.917412,0.966232,9.715615e-01,9.715615e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,0.852990,0.868696,0.945965,0.937717,0.897681,0.919547,0.883834,0.852265,0.930903,0.898327,...,0.958305,0.929433,0.915758,0.615376,0.961906,0.000000,0.905733,0.894661,9.272764e-01,9.272764e-01
621,0.901102,0.930068,0.944117,0.952347,0.917412,0.879656,0.874755,0.900614,0.935788,0.915730,...,0.922882,0.870197,0.937818,0.852228,0.901795,0.905733,0.000000,0.922538,9.061390e-01,9.061390e-01
622,0.950424,0.851514,0.950632,0.966035,0.966232,0.928669,0.933159,0.950179,0.961913,0.953489,...,0.957542,0.940756,0.933961,0.911814,0.945811,0.894661,0.922538,0.000000,3.220534e-01,3.220534e-01
623,0.940666,0.873019,0.916042,0.972255,0.971561,0.954007,0.943593,0.945982,0.957721,0.956936,...,0.953326,0.903103,0.937640,0.828208,0.967834,0.927276,0.906139,0.322053,0.000000e+00,2.220446e-16


### Поиск вакансии с наибольшим числом похожих ###

In [7]:
vac['same'] = vac.apply(lambda d: (d<0.5).sum(), axis=1)
for index in vac[vac.same==vac.same.max()].index:
    print(data.iloc[index, 0])
    print('_'*120)

Разработчик-аналитик машинного обучения в Еду🍎

Яндекс.Еда меняет привычки людей в сфере питания, развивает фудтех и буквально приближает будущее. Наша команда помогает улучшить Еду с помощью алгоритмов и больших данных. Мы разрабатываем рекомендации ресторанов и магазинов, товаров и блюд, прогнозируем время готовки и доставки заказа, потребность в курьерах.
У нас глобальные и смелые планы и, возможно, именно вы поможете нам осуществить их как можно скорее.

Мы ждем, что вы:
• знаете методы анализа данных и ML, применяли их на практике
• знаете матстат и тервер
• пишете на Python или C++

Что нужно делать:
• отвечать за все стадии ML-проектов: от формализации задачи до внедрения в продакшен
• находить точки роста и улучшать бизнес-показатели
• выдвигать гипотезы и проверять их экспериментально

Будет плюсом, если вы:
• внедряли модели ML в продакшен
• разрабатывали рекомендательные системы
• проводили эксперименты и анализировали результаты А/Б-тестов

https://ya.cc/t/RMAyiiBYSxFw4

#M